<a href="https://colab.research.google.com/github/LennyJS/AIS2022/blob/main/FinalProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Final Project

In [8]:
!pip install sentence_transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 10.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 105.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 74.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 24.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 111.3 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=78f6859a23b3167f3e1df8f5cd586a7ab48532fc9c5843816fff65d370deefd4
  Stored in directory: /root/.cache/pip/wheels/5e/6f/8c/d88aec621f3f542d26fac0342bef5e693335d125f4e54aeffe
Successfully built sentence_transformers


In [9]:
import numpy as np
import pandas as pd
import glob, os
from google.colab import drive
import xml.etree.ElementTree as ET
import parser
from pandas.core.api import notnull

In [10]:
# since files are in google drive a mount is needed
drive.mount('/content/gdrive/', force_remount=True)

Mounted at /content/gdrive/


In [ ]:
# creation of dataframe with csv file from google drive
df = pd.read_csv("/content/gdrive/MyDrive/Firmen_DF1.csv", index_col = 0)

# create dataframe were column 'Gegenstand' is not null and safe this in df_filtered
df_filtered = df[df['Gegenstand'].notnull()]

In [ ]:
for col in df:
  print(col)

In [ ]:
df_filtered = df[df['Bezeichnung_Aktuell_Firma 61'].notnull()]

In [ ]:
df_filtered.loc[8998].tolist()

In [ ]:
col_list = []
for col in df_filtered:
  if col.startswith("Bezeichnung_Aktuell_Firma"):
    col_list.append(col)

In [ ]:
df_filtered2 = df_filtered[col_list]

In [ ]:
check_list = df_filtered2.loc[8998].tolist()

In [ ]:
df_filtered = df[df['Bezeichnung_Aktuell_Firma 0'].notnull()]

In [ ]:
doppelteinträge = []
for ind in df.index:
  forstring = df["Bezeichnung_Aktuell_Firma 0"][ind]
  if forstring in check_list:
    doppelteinträge.append(forstring)

In [ ]:
doppelteinträge

['Unternehmertum VC Fonds III GmbH & Co. KG']

# 3. Normalform der Daten:

Entfernen der nicht benötigten Spalten: (alle Firmendaten, die nicht index 0 1 haben) 

In [ ]:
df_norm = df

In [ ]:
firmen_cols = ["ID", "Name", "Ort", "Postleitzahl", "Straße", "Hausnummer", "Gegenstand", "Stammkapital", "Gesellschaftsform"]

firmen_df = pd.DataFrame(columns = firmen_cols)

id_counter = 1


for ind in df_norm.index:
  name = df_norm["Bezeichnung_Aktuell_Firma 0"][ind]
  ort = df_norm["Ort_Firma 0"][ind]
  plz = df_norm["Postleitzahl_Firma 0"][ind]
  straße = df_norm["Strasse_Firma 0"][ind]
  hausnummer = df_norm["Hausnummer_Firma 0"][ind]
  gegenstand = df_norm["Gegenstand_oder_Geschaeftszweck 0"][ind]
  stammkapital = df_norm["Zahl 0"][ind]
  gesellschaftsform = df_norm["content_Firma 0"][ind]


  df_norm["Bezeichnung_Aktuell_Firma 0"][ind] = id_counter

  iter_list = [id_counter, name, ort, plz, straße, hausnummer, gegenstand, stammkapital, gesellschaftsform]
  firmen_df.loc[len(firmen_df)] = iter_list

  id_counter += 1

<ipython-input-129-fd947f55e86e>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_norm["Bezeichnung_Aktuell_Firma 0"][ind] = id_counter


In [ ]:
col_list = []
for col in df_filtered:
  if col.startswith("Bezeichnung_Aktuell_Firma") | col.startswith("Ort_Firma") | col.startswith("Postleitzahl_Firma") | col.startswith("Strasse_Firma") | col.startswith("Hausnummer_Firma") | col.startswith("Gegenstand_oder_Geschaeftszweck") | col.startswith("Zahl") | col.startswith("content_Firma"):
    col_list.append(col)

In [ ]:
col_list.remove("Bezeichnung_Aktuell_Firma 0")
df_norm = df.drop(col_list, axis = 1)
df_norm = df_norm.rename(columns = {"Bezeichnung_Aktuell_Firma 0": "Firmen_Id"})

In [ ]:
firmen_df.head(1)

,ID,Name,Ort,Postleitzahl,Straße,Hausnummer
0,1,09 Media GmbH,Regensburg,93059.0,Im Gewerbepark,A10


Durch den DataFrame iterieren und Personen IDs für nicht gelistete Personen erstellen.
Neuen DataFrame aufsetzen, dass die Kombination der Firmen und PersonenIds wiedergibt.

In [ ]:
personen_cols = ["ID", "Vorname", "Nachname", "Ort", "Geburtsdatum"]
personen_df = pd.DatFrame()

for ind in df_norm.index:

  personen_count = 0
  id_counter = 0

  while df_norm["Nachname " + str(personen_count)][ind].notnull():


    if df_norm["Vorname" + str(personen_count)][ind].notnull():
      vorname = df_norm["Vorname" + str(personen_count)][ind]

    personen_count += 1

In [ ]:
for col in df_norm:
  print(col)

# Domis völlig unnötiger Tiefensuchansatz


4 Übergabeargumente:


1.   Child: XML Tree der Übergeben werden soll
2.   List: XML Calls in Baumstruktur
3.   Dict: Dict zum Erstellen des Dataframes
4.   Count: Zum Hochzählen der gleichen Keys



In [ ]:
def XML_to_DataFrame(Child, List, Dict, Count, Affix):
  for Item in List:
    if isinstance(Item, str):
      for key in Dict.keys():
        if Affix != 0:
          item_C = Item + Affix + ' ' + str(Count)    
        else:
          item_C = Item + ' ' + str(Count) 
        if key == item_C:
          Count += 1
      if Child.find(basic_URL + Item) is not None:
        if Child.find(basic_URL + Item).text is not None:
          if Affix != 0:
            Dict.update({Item + Affix + ' ' +str(Count):Child.find(basic_URL + Item).text})    
          else:
            Dict.update({Item + ' ' +str(Count):Child.find(basic_URL + Item).text}) 
    elif isinstance(Item, list):
      for key in Item[0]:
        for child_Iter in Child.findall(basic_URL + Item[0][key]):
         XML_to_DataFrame(child_Iter, Item[1], Dict, Count, key)

In [ ]:
#basicUrl
basic_URL = ".//{http://www.xjustiz.de}"

Firma_List = [[{"_Firma":"Rechtsform"}, ["content"]], [{"_Firma":"Anschrift"}, ["Strasse","Hausnummer","Postleitzahl","Ort"]],"Bezeichnung_Aktuell"]
Firma = [{"_Firma":"Organisation"}, Firma_List]

Beteiligte_List = [[{0:"rollenbezeichnung"},["content"]],[{0:"Natuerliche_Person"},["Vorname","Nachname","Beruf", "Ort"]]]
Beteiligte = [{0:"Beteiligter"}, Beteiligte_List]

Stammkapital = [{0:"Stammkapital"}, ["Zahl"]]

XML_List = ["Gegenstand_oder_Geschaeftszweck", Firma, Beteiligte, Stammkapital]

In [ ]:
XML = ".//{http://www.xjustiz.de}Gegenstand_oder_Geschaeftszweck"
path = "/content/gdrive/MyDrive/PDS/a/"
df = pd.DataFrame()

for subdir, dirs, files in os.walk(path):
    for file in files:
        filepath = subdir + os.sep + file

        if filepath.endswith("xml"):
          try:
            tree = ET.parse(filepath)
            Dict = dict()
            XML_to_DataFrame(tree, XML_List, Dict, 0, 0)
            df = pd.concat([df,pd.DataFrame(Dict, index=[0])], axis=0, ignore_index=True)
            
          except Exception as e: 
            print(e)

not well-formed (invalid token): line 1, column 559
not well-formed (invalid token): line 1, column 541
not well-formed (invalid token): line 1, column 561
not well-formed (invalid token): line 1, column 544
not well-formed (invalid token): line 1, column 567
not well-formed (invalid token): line 1, column 538
not well-formed (invalid token): line 1, column 539
not well-formed (invalid token): line 1, column 549
not well-formed (invalid token): line 1, column 573
not well-formed (invalid token): line 1, column 579
not well-formed (invalid token): line 1, column 553
not well-formed (invalid token): line 1, column 536
not well-formed (invalid token): line 1, column 543
not well-formed (invalid token): line 1, column 563
not well-formed (invalid token): line 1, column 545


In [ ]:
df

In [ ]:
df_filtered = df[df['Gegenstand_oder_Geschaeftszweck 0'].notnull()]

In [ ]:
df_filtered.shape

(7191, 1290)

# OLD wie LArs


In [ ]:
#basicUrl
basic_URL = ".//{http://www.xjustiz.de}"

#gegenstand
gegenstand = "Gegenstand_oder_Geschaeftszweck"

#beteiligter
beteiligter = "Beteiligter"

#beteiligter1 -> firmenname
bezeichnung_aktuell = "Bezeichnung_Aktuell"

#beteiligter1 -> rechtsform
rechtsform = "Rechtsform"
#rechtform -> content
rechtsform_string = "content"

#beteiligter1 ->anschrift
anschrift = "Anschrift"

#anschrift -> strasse
strasse = "Strasse"
#anschrift -> hausnummer
hausnummer = "Hausnummer"
#anschrift -> plz
plz = "Postleitzahl"
#anschrift -> ort
ort = "Ort"


#beteiliger* -> vorname
vorname = "Vorname"
#beteiliger* -> nachname
nachname = "Nachname"
#beteiliger* -> beruf
beruf_person = "Beruf"

#Stammkapital
stammkapital = "Stammkapital"
#Stammkapital -> Zahl
euro = "Zahl"


In [ ]:
def XML_to_Text(child, URL):
  node = child.find(URL)
  if node is not None:
      return node.text
  else:
      return np.NaN

In [ ]:
def XML_to_DF(tree):

  Dict = ({gegenstand:XML_to_Text(tree, basic_URL + gegenstand)})
  # tree.find(basic_URL + gegenstand).text
  beteiligter_count = 1  

  for beteiligter_iter in tree.findall(basic_URL + beteiligter):
   

    if beteiligter_count == 2:
      beteiligter_count += 1
      continue


    #Erster Beteiligter
    if beteiligter_count == 1:
      
      Dict.update({bezeichnung_aktuell:XML_to_Text(beteiligter_iter, basic_URL + bezeichnung_aktuell)})
      #Dict.update({bezeichnung_aktuell:beteiligter_iter.find(basic_URL + bezeichnung_aktuell).text})

      for rechtsform_iter in beteiligter_iter.findall(basic_URL + rechtsform):

        Dict.update({rechtsform_string:XML_to_Text(rechtsform_iter, basic_URL + rechtsform_string)})
        #Dict.update({rechtsform_string:rechtsform_iter.find(basic_URL + rechtsform_string).text})

      for anschrift_iter in beteiligter_iter.findall(basic_URL + anschrift):

        Dict.update({strasse:XML_to_Text(anschrift_iter, basic_URL + strasse)})
        Dict.update({hausnummer:XML_to_Text(anschrift_iter, basic_URL + hausnummer)})
        Dict.update({plz:XML_to_Text(anschrift_iter, basic_URL + plz)})
        Dict.update({ort:XML_to_Text(anschrift_iter, basic_URL + ort)})
      
      beteiligter_count = beteiligter_count + 1
      continue


    #Alle Personen
    else:

      Dict.update({vorname + str(beteiligter_count - 2):XML_to_Text(beteiligter_iter, basic_URL + vorname)})
      Dict.update({nachname + str(beteiligter_count - 2):XML_to_Text(beteiligter_iter, basic_URL + nachname)})
      Dict.update({ort + str(beteiligter_count - 2):XML_to_Text(beteiligter_iter, basic_URL + ort)})
      Dict.update({beruf_person + str(beteiligter_count - 2):XML_to_Text(beteiligter_iter, basic_URL + beruf_person)})
  

      beteiligter_count += 1
      #if(beteiligter_count > 100):
        #print(beteiligter_count)

  

  
  for stammkapital_iter in tree.findall(basic_URL + stammkapital):
    Dict.update({euro:XML_to_Text(stammkapital_iter, basic_URL + euro)})

    #Dict.update({euro:stammkapital_iter.find(basic_URL + euro).text})

  return Dict

In [ ]:
tree = ET.parse("/content/gdrive/MyDrive/PDS/a/automaten-profis-g_steinfurt_hrb_14132/automaten-profis-g_steinfurt_hrb_14132_SI_1_1669935176726.xml")
test_df = pd.DataFrame()
Dict = XML_to_DF(tree)

test_df = pd.concat([test_df,pd.DataFrame(Dict, index=[0])], axis=0, ignore_index=True)
test_df

,Gegenstand_oder_Geschaeftszweck,Bezeichnung_Aktuell,content,Strasse,Hausnummer,Postleitzahl,Ort,Vorname1,Nachname1,Ort1,Beruf1,Zahl
0,Die Beratung und der Vertrieb von Food-Automat...,Automaten-Profis GmbH,Gesellschaft mit beschränkter Haftung,Am Goldberg,13,48432,Rheine,Ümit,Yildiz,Greven,None,25000.00


In [ ]:
XML = ".//{http://www.xjustiz.de}Gegenstand_oder_Geschaeftszweck"
path = "/content/gdrive/MyDrive/PDS/a/"
df = pd.DataFrame()

for subdir, dirs, files in os.walk(path):
    for file in files:
        filepath = subdir + os.sep + file

        if filepath.endswith("xml"):
          try:
            tree = ET.parse(filepath)
            Dict = XML_to_DF(tree)
            df = pd.concat([df,pd.DataFrame(Dict, index=[0])], axis=0, ignore_index=True)
            
          except Exception as e: 
            print(e)
            #print(filepath)

not well-formed (invalid token): line 1, column 559
not well-formed (invalid token): line 1, column 541
not well-formed (invalid token): line 1, column 561
not well-formed (invalid token): line 1, column 544
not well-formed (invalid token): line 1, column 567
not well-formed (invalid token): line 1, column 538
not well-formed (invalid token): line 1, column 539
not well-formed (invalid token): line 1, column 549
not well-formed (invalid token): line 1, column 573
not well-formed (invalid token): line 1, column 579
not well-formed (invalid token): line 1, column 553
not well-formed (invalid token): line 1, column 536
not well-formed (invalid token): line 1, column 543
not well-formed (invalid token): line 1, column 563
not well-formed (invalid token): line 1, column 545


In [ ]:
df

,Gegenstand_oder_Geschaeftszweck,Bezeichnung_Aktuell,content,Strasse,Hausnummer,Postleitzahl,Ort,Vorname1,Nachname1,Ort1,...,Ort158,Beruf158,Vorname159,Nachname159,Ort159,Beruf159,Vorname160,Nachname160,Ort160,Beruf160
0,None,Axel Römhildt Vermögens und Grundstücks GmbH &...,Kommanditgesellschaft,Keltenstraße,15,56332,Dieblich,NaN,NaN,Koblenz,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"Ausführen von Personaldienstleistungen, gewerb...",AXIMA Personallogistik und Dienstleistungen He...,Gesellschaft mit beschränkter Haftung,Bäckerstr. 17 -,19,32052,Herford,Bedir,Caliskan,Hövelhof,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"Der Groß- und Einzelhandel, Import und Export,...",Aytaq Marketing GmbH,Gesellschaft mit beschränkter Haftung,Gottlieb-Daimler-Str.,14,68165,Mannheim,Yücel,Kalin,Lanaken/Belgien,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,None,AXP Consulting GmbH & Co. KG,None,Torgauer Str.,233,04347,Leipzig,NaN,NaN,Leipzig,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Die Beratung in Steuerangelegenheiten.,Axel Bahr & Partner PartG mbB Steuerberater,Partnerschaft,Mittelstraße,5,58285,Gevelsberg,NaN,NaN,Sprockhövel,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
848,NaN,1. KMPS UG (haftungsbeschränkt) & Co. KG,Kommanditgesellschaft,Friedrichsorter Straße,37,24214,Gettorf,NaN,NaN,Gettorf,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
849,None,1. Koop + Lohmann Solarpark GmbH & Co. KG,None,Kuppendof,56,27245,Kirchdorf,Thomas,Lohmann,Köln,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
850,Anbietung von Dienstleistungen in den Bereiche...,09 Media GmbH,Gesellschaft mit beschränkter Haftung,Im Gewerbepark,A10,93059,Regensburg,Michael,Simmendinger,Riekofen,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
851,Gegenstand des Unternehmens ist die Durchführu...,10 k Transfer GmbH,Gesellschaft mit beschränkter Haftung,Stuhlbrudergasse,5,67346,Speyer,Silvia,Kohlberger,Schwegenheim,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Clustering


In [ ]:
# Perform kmean clustering
corpus = df_filtered["Gegenstand_oder_Geschaeftszweck 0"].tolist()
corpus_embeddings = embedder.encode(corpus, show_progress_bar=True)
num_clusters = 9
clustering_model = KMeans(n_clusters=num_clusters)
clustering_model.fit(corpus_embeddings)
cluster_assignment = clustering_model.labels_

clustered_sentences = [[] for i in range(num_clusters)]
for sentence_id, cluster_id in enumerate(cluster_assignment):
    clustered_sentences[cluster_id].append(corpus[sentence_id])


Batches:   0%|          | 0/225 [00:00<?, ?it/s]

In [ ]:
for i, cluster in enumerate(clustered_sentences):
    print("Cluster ", i+1)
    print(cluster)
    print("")

In [ ]:
clustered_sentences[5]

In [ ]:
Agrarwirtschaft = ['Reparaturen an Bau- und Landmaschinen',
                   ]

Bau = ['Holz- und Bautenschutz, Abdichtung und Sanierung an Gebäuden.','Bauleistungen im Bereich des Hochbaus','das Projektmanagement im Bauwesen.',
       'Die Durchführung sämtlicher Tätigkeiten, Aufgaben und Dienstleistungen die im Zusammenhang stehen, die ein Putz- und Stuckateurbetrieb mit sich bringt; darüberhinaus die Ausübung sämtlicher Tätigkeiten die bei der Handwerkskammer unter B1 und B2 geführt werden.',
       'Die Durchführung von Bauleistungen im Bereich des Hoch- und Tiefbaus, insbesondere von Maurer-, Beton- und Stahlbetonarbeiten, Zimmermannsarbeiten, Dachdeckerarbeiten und auch die Errichtung von schlüsselfertigen Ein- und Mehrfamilienhäusern.',
       'Durchführung von Trockenbauarbeiten.',
       'Die Erbringung von Maurerarbeiten und Betonbau sowie aller damit im Zusammenhang stehenden Tätigkeiten.',
       'Erstellung von Arbeits- und Schutzgerüsten.',
       'Baggerbetrieb sowie Durchführung von Erdbewegungen.',
       'Begutachtung von Betonschäden.',
       'Erstellung von Rohbauten im Baugewerbe, speziell für Maurer- und Betonarbeiten sowie die Reinigung von Gebäuden und Baustellen.',
       'Reparaturen an Bau- und Landmaschinen',
       'der Hochbau sowie alle damit zusammenhängenden Leistungen.',
       'Gegenstand des Unternehmens ist die Vermietung und der Handel von neuen und gebrauchten, eigenen und angemieteten Baumaschinen, sowie die Erbringung von Transportleistungen für vermietete und andere Maschinen, sowie Fremdtransporte für Dritte. Die Gesellschaft ist berechtigt, Zweigniederlassungen zu errichten und sich an anderen Unternehmen, insbesondere als persönlich haftende Gesellschafterin zu beteiligen.',
        'Stahl- und Metallbau sowie Handel mit Baustoffen jeglicher Art und die damit verbundenen Dienstleistungen.',
       
       ]

Chemie_Rohstoffe = ['Planung von Laborräumen im In- und Ausland',]

Dienstleistungen_Handwerk = ['Die Durchführung sämtlicher Tätigkeiten, Aufgaben und Dienstleistungen die im Zusammenhang stehen, die ein Putz- und Stuckateurbetrieb mit sich bringt; darüberhinaus die Ausübung sämtlicher Tätigkeiten die bei der Handwerkskammer unter B1 und B2 geführt werden.',
                             'Die Durchführung von Bauleistungen im Bereich des Hoch- und Tiefbaus, insbesondere von Maurer-, Beton- und Stahlbetonarbeiten, Zimmermannsarbeiten, Dachdeckerarbeiten und auch die Errichtung von schlüsselfertigen Ein- und Mehrfamilienhäusern.',
                             'ein Malermeisterbetrieb',
                             'Dienstleistungen im Bau- und Projektmanagement.',
                             'Der Betrieb eines Unternehmens für Garten- und Landschaftsbau und artverwandte Tätigkeiten.',]

Internet = ['Angewandte Forschung und Entwicklung in der Informationstechnik, insbesondere Software-Entwicklung.',
            'IT- Dienstleistungen, Verkauf von Software und die Vermittlung der Speditionsleistungen.',
            'Beratung von Unternehmen und Institutionen bei Themen des digitalen Wandels, insbesondere die Nutzung von Netzintelligenz, die Entwicklung von diesbezüglichen Softwarelösungen und deren Vertrieb, Trainings- sowie die Prozess- und Organisationsberatung.',
            'Implementierung von kundenspezifischer Software, insbesondere die Entwicklung von Onlineshops und Unternehmenswebseiten, sowie die Erbringung von Dienstleistungen und der Handel im Bereich Marketing und IT',
            'IT-Beratung jeglicher Art sowie der Handel mit IT-Geräten und Software jeglicher Art wie aus die Herstellung von IT-Geräten',
            'Der Vertrieb von IT Hardware, Software sowie Planung, Beratung, Konzeption, Installation von IT-Infrastrukturen und alle hiermit in Zusammenhang stehenden Tätigkeiten.',
            'Gegenstand des Unternehmens der Gesellschaft ist die Herstellung, der Vertrieb und die Vermarktung von Softwareprodukten und Hardware für mobile Plattformen, sowohl auf nationaler als auch auf internationaler Ebene.',
            'Die Entwicklung, Produktion und Vertrieb von IT-Lösungen im Bereich integrieter mulitfunktionaler Straßenbeleuchtung und anderen derartigen Stadtmöbeln. Die IT-Lösungen generieren dabei Daten, die wiederum vermarktet werden. Darüber hinaus bietet das Unternehmen stratgische Beratung in genau diesen Geschäftsfeldern an.',
            'Die Herstellung, Wartung, der Vertrieb und der Handel von Computer-Software, der Handel mit Computerhardware und Zubehör, elektrischen und elektronischen Geräten und Anlagen zur         Telekommunikation, die Beratung Dritter zur Informationstechnologie sowie der Betrieb einer   Werbeagentur. Genehmigungspflichtige Geschäfte sind ausdrücklich ausgeschlossen.',
            'Social Media und Internet-Marketing Management/ Beratung',
            ]

Energie_Umwelt = ['Entwicklung und Herstellung von technischen Anlagen, Maschinen, Maschinenteilen und Zubehör sowie der Handel und Vertrieb von solchen Gegenständen, insbesondere für die Wassertechnik und Umweltschutz.',
                  'Beratung, Planung und Ausrüstung von Wasserkraftanlagen.',
                  'Herstellung und Vertrieb von Kolbenwerkstoffen und Antriebsaggregaten.',
                  'Errichtung und Betrieb einer Biogasanlage, Erzeugung und Verwertung von Strom und Wärme aus Biomasse sowie Versorgung von Weiterverteilern mit Strom und Wärme.',
                  'Erwerb und Verwaltung von Waldflächen',
                  'Die Beteiligung und Verwaltung von Beteiligungen an Unternehmen aller Art sowie die Erbringung von Beratungsleistungen im energiewirtschaftlichen Umfeld.',
                  'Gegenstand des Unternehmens ist der Erwerb und das Halten von Gesellschaftsanteilen an Biogasprojekten aller Art. In dieser Funktion betreibt die Gesellschaft selber kein operatives Geschäft, sondern gründet und hält nach Gründung diese Gesellschaftsanteile an den sogenannten Projekten.',
                  ]

                
Finanzen_Versicherungen_Immobilien = ['Schaffung, Verwaltung, Bewirtschaftung und Verwertung von Haus- und Grundeigentum',
                                      'Erwerb und Verwertung von Grundstücken sowie Erwerb, Herstellung und Verwertung von Häusern.',
                                      'Der An- und Verkauf sowie die Verwaltung eigener Immobilien',
                                      'Der An- und Verkauf, die Vermietung und Verwaltung von Immobilien',
                                      'Der An- und Verkauf, die Vermietung und Verwaltung von Immobilien',
                                      'Projektierung und Entwicklung von Bauobjekten sowie Vermittlung, Kauf und Verkauf von Immobilien',
                                      'Erwerb von Immobilien, deren Vermietung und Verwaltung',
                                      'Erwerb und die Veräußerung von bebautem und unbebautem Grundbesitz sowie Wohn- und Teileigentum für eigene Rechnung.',
                                      'Verwaltung von Immobilien',
                                      'Sanierung, Erwerb, Verkauf und Verwaltung von Immobilien sowie Bauträgertätigkeiten gemäß § 34 c GewO.',
                                      'Kauf und Verkauf von und Vermittlung des Abschlusses von Verträgen oder Nachweis der Gelegenheit zum Abschluss von Verträgen über Grundstücke, grundstücksgleiche Rechte, gewerbliche Räume und Wohnräume und  Erbringung von Trockenbauarbeiten und Maurerarbeiten',
                                      'Gegenstand des Unternehmens ist es, gewerbsmäßig \n- Bauvorhaben als Bauherr im eigenen Namen für eigene oder fremde Rechnung vorzubereiten oder durchführen zu lassen und dazu Vermögenswerte von Erwerbern, Mietern, Pächtern oder sonstigen Nutzungsberechtigten oder von Bewerbern um Erwerbs- oder Nutzungsrechte zu verwenden, \n- Bauvorhaben als Baubetreuer im fremden Raum für fremde Rechnung wirtschaftlich vorzubereiten oder durchzuführen, \n- den Abschluss von Verträgen über Grundstücke, grundstücksgleiche Rechte, gewerbliche Räume oder Wohnräume zu vermitteln oder die Gelegenheit zum Abschluss solcher Verträge nachzuweisen, \n- ferner das Halten und Verwalten von Beteiligungen, die Übernahme von Geschäftsführungsaufgaben, \n- Vermögensverwaltung.',
                                      'Der Erwerb, die Veräußerung und die Verwaltung von Immobilien. Geschäfte nach § 34 c Abs. 1 Nr. 1 und 4 GewO sind ausdrücklich nicht Gegenstand des Unternehmens.',
                                      'Vermittlung von Immobilien und Unternehmensbeteiligungen',
                                      'Die Müll- und Abfallsortierung, der Hausmeisterservice und der An- und Verkauf von Immobilien sowie die Erbringung von Dienstleistungen und Bauleistungen im Zusammenhang mit der Bewirtschaftung und Sanierung/Renovierung von Immobilien.',
                                      'Gegenstand des Unternehmens sind Unternehmensberatungs- und Verwaltungstätigkeiten. Die Gesellschaft ist außerdem zur Übernahme der Geschäftsführung anderer Gesellschaften, insbesondere von Gesellschaften in Form der Kommanditgesellschaft als persönlich haftende Gesellschafterin berechtigt. Ausgenommen sind erlaubnispflichtige Tätigkeiten.',
                                      'Die Beteiligung und Verwaltung von Beteiligungen an Unternehmen aller Art sowie die Erbringung von Beratungsleistungen im energiewirtschaftlichen Umfeld.',
                                      'Erwerb von, die Beteiligung an und die Geschäftsführung bei anderen Gesellschaften.',
                                      'Verwaltung von Unternehmen; Unternehmens- und Geschäftsführungsberatungsdienste; Beratung und Hilfe bezüglich der Betriebs-, Geschäfts- oder Unternehmensführung sowie der Betriebs-, Geschäfts- oder Unternehmensorganisation; Hilfe in Geschäftsangelegenheiten; Geschäftsführung; administrative Dienstleistungen und Verpachtung und Vermietung von Immobilien; Mergers- und Akquisitionsgeschäfte, nämlich finanzielle Beratung beim Kauf oder Verkauf von Unternehmensbeteiligungen, Transport- und Lagerdienstleistungen. Erzeugung von Energie, insbesondere mittels Photovoltaikanlagen. Verwaltung und Lizenzierung von gewerblichen Schutzrechten.',
                                      'die Übernähme der Geschäftsführung von anderen Unterenehmen sowie der Erwerb und die Verwaltung von Beteiligungen an Unternehmen insbesondere an Unternehmen, deren Gegenstand die Herstellung von und der Handel mit Getränken aller Art ist.',
                                      'ist der Erwerb und die Verwaltung von Beteiligungen insbesondere an Unternehmen der kommunalen Wirtschaft der Stadt Velbert sowie die Förderung dieser Unternehmen.',
                                      ]

Metall_Elektronik = ['Das Erbringen von administrativen, kaufmännischen und technischen Dienstleistungen für Unternehmen, die insbesondere die Entwicklung und Erprobung von Motoren, Motorenbauteilen und Betriebsstoffen mit Motorenprüfständen, die Entwicklung und Erprobung von Motorbaugruppen und Betriebsstoffen für Kraftfahrzeuge sowie die Durchführung von Forschungs- und Entwicklungsarbeiten mit Einrichtungen für Bauteilversuche und Festigkeitsnachweise sowie Durchführung von weiteren Versuchs- und Entwicklungsdienstleistungen für die Automobil-, Automobilzulieferer-, Luftfahrt- und Kraftwerksindustrie und andere Industriezweige zum Gegenstand haben; die Beteiligung an diesen Unternehmen im eigenen Namen und auf eigene Rechnung, nicht für Dritte, und die Verwaltung derselben.',
                     'Handel mit Motorrädern, Motoradteilen und Motorradzubehör und -bekleidung aller Art, sowie der Betrieb einer Motorrad-Werkstatt.',
                     'Der Betrieb einer Kraftfahrzeug- und Lastkraftwagen-Reparaturwerkstatt sowie ferner der Handel mit Kraftfahrzeugen und Lastkraftwagen, Ersatzteilen und Zubehör und Kraftfahrzeugölen.',
                     'Stahl- und Metallbau sowie Handel mit Baustoffen jeglicher Art und die damit verbundenen Dienstleistungen.',
                     

                     ]

Freizeit = ['Das Betreiben eines Spielcasinos mit Bewirtung.',
            'ist der Betrieb von Spielhallen.',
            'Betrieb von Sportstätten mit angeschlossener Gastronomie, insbesondere Kletteranlagen und Fitnessstudios, Durchführung von Sportveranstaltungen sowie Beratung im Zusammenhang mit der Durchführungvon Sportveranstaltungen und des Sportstättenbetriebes.',
            'der Betrieb eines Fitness-Studios.',
            'Der Betrieb einer Yogaschule und eines Yogastudios.',

            ]

ÖffentlicherSektor_Bildung = ['1.Betrieb von Bildungsstätten für die berufliche Bildung in der Wirtschaftsregion Leipzig. 2. Die Gesellschaft hat die Aufgabe die berufliche Bildung sowie berufsorientierenden Maßnahmen für Jugendliche auf der Grundlage des regionalen Bedarfs in Kooperation mit Betrieben und Instituten zu organisieren und durchzuführen. 3. Die Gesellschaft ist auch tätig für die Durchführung von Bildungsmaßnahmen sowie wissenschaftlicher und allgemeinbildender Veranstaltungen für die Bevölkerung. Die Gesellschaft darf darüber hinaus überregionale und internationale Maßnahmen und Kooperationen der beruflichen Bildung durchführen.',
                              'Zweck der Genossenschaft ist die Förderung ihrer Mitglieder vorrangig  durch eine gute, sichere und sozial verantwortbare Wohnungsversorgung .\nDie Genossenschaft kann Bauten in allen Rechts- und Nutzungsformen bewirtschaften, errichten, erwerben, vermitteln und betreuen. Sie kann alle im Bereich der Wohnungs- und Immobilienwirtschaft, des Städtebaus und der Infrastruktur anfallenden Aufgaben übernehmen.\nHierzu gehören Gemeinschaftsanlagen und Folgeeinrichtungen, Läden und Räume für Gewerbebetriebe, soziale, wirtschaftliche und kulturelle Einrichtungen und Dienstleistungen. Beteiligungen sind zulässig.',
                              'Die Weiterbildung im Bereich Neuropsychologie und Gesundheitswesen.',

                              
                              ]

Handel_Konsum = ['Erfassung und Aufbereitung fleischlicher Rohstoffe.','Vertrieb und Wartung von Toranlagen.',
                 'Der Handel (einschließlich Export und Import) mit Konsumwaren insbesondere im Bereich Haushalt und Unterhaltung, sowie das Vermarkten derartiger Artikel und sonstiger nicht genehmigungspflichtiger Vermittlungs- und Beratungsgeschäfte u.a. im Bereich Windkraft, als auch die Land- und Forstwirtschaft.',
                 'Betrieb einer Bäckerei und der Handel mit Backwaren.',
                 'Der Einkauf und der Verkauf von Kosmetikwaren aller Art, Schulung, Lagerhaltung und Distribution.',
                 'An- und Verkauf von Textilien und modischem Zubehör.',
                 'der Großhandel im Sanitär-, Heizungs- und Lüftungsbereich sowie in artverwandten Bereichen.',
                 'Großhandel mit Materialien zur Raumausstattung.',
                 'der Handel und Verkauf von Textilien aller Art und der Betrieb einer Handelsagentur auf diesem Gebiet',
                 'Handel mit Lebensmittelspezialitäten jeder Art einschließlich Im- und Export.',
                 'Der An- und Verkauf von Maschinen und Anlagen jeglicher Art, neu und gebraucht nebst allen damit zusammenhängenden Geschäften.',
                 'Handel mit Möbeln, Accessoires, Lampen, Stoffen, Farben und Spielzeug sowie Dienstleistungen im Bereich Einrichtung.',
                 'Der Einzelhandel mit Küchen, Möbeln und Elektrogeräten.',
                 'Import und Vertrieb von Konsumgütern über verschiedene Online Marktplätze, inklusive Amazon: Spielzeug, Sport- und Freizeitartikel, Baumarktartikel, Büro-und Schreibwaren, Drogerieartikel, Gartenartikel, Haustierartikel und Haushaltsartikel.', 
                 ]

Pharma_Gesundheit = ['Planung von Laborräumen im In- und Ausland',
                     'Der Krankentransport, Krankenfahrten mit Mietwagen und die Erbringung von medizinischen Dienstleistungen bzw. Kurierdienste die keiner behördlichen Genehmigung bedürfen.',
                     'Die Weiterbildung im Bereich Neuropsychologie und Gesundheitswesen.',
                     'die Beratung und das Betreiben von Krankenhäusern oder Krankenhaus-Dienstleistern auf qualitativ hohem Niveau, soweit dies nicht genehmigungspflichtig ist.',
                     'der Handel mit medizinischen Geräten.',
                     'Der Betrieb von Einrichtungen zur ambulanten und stationären Erbringung von Rehabilitationsdienstleistungen und sonstigen Gesundheits- und Beratungsdienstleistungen in Zusammenarbeit mit Kliniken und als Eigenbetriebe, überwiegend im Raum Recklinghausen.',
                     

                     ]

Tourismus_Gastronomie = ['der Betrieb eines Hotels unter der Bezeichnung "City House" in Hamburg sowie sämtliche damit verbundenen Tätigkeiten einer juristischen Person, soweit diese keiner Erlaubnis bedürfen.\n',
                         'die Veranstaltung und Vermittlung von Reisen und weltweite Incentives.',

                         
                         
                         ]

Verkehr_Logistik = ['Erbringung von Eisenbahnverkehrsleistungen im Güter- und Personenverkehr',
                    'Fortführung des bisher von der Firma Michael Schirra OHG betriebenen Unternehmens, insbesondere die Personenbeförderung mit Kraftfahrzeugen im Linien- und Reiseverkehr',
                    'Dienstleistungen im Bereich des Gütertransports, insbesondere im Schienenverkehr, sowie sämtliche damit im Zusammenhang stehenden Geschäfte.',
                    'Organisieren und Durchführen von Transporten und Verpackungen jeglicher Art sowie einfachen produktionsnahen Dienstleistungen.',
                    'Der Krankentransport, Krankenfahrten mit Mietwagen und die Erbringung von medizinischen Dienstleistungen bzw. Kurierdienste die keiner behördlichen Genehmigung bedürfen.',
                    'Erarbeitung und Entwicklung von Logistiklösungen für alle Unternehmensbereiche im Transportwesen, Entwicklung und Vertrieb von Logistiksoftware insbesondere für Transportvermittlung, Kurierdienst und Spedition; Erwerb und Verwaltung von Beteiligungen sowie die Übernahme der persönlichen Haftung und der Geschäftsführung bei Handelsgesellschaften, insbesondere die Beteiligung als persönlich haftende geschäftfsführende Gesellschafterin an der Gesellschaft in Firma SKLM Sächsisches Kurier- und Logistikzentrum GmbH & Co. KG mit dem Sitz in Dresden',
                    'Der Betrieb eines Taxiunternehmens.',
                    'Post-, Kurier- und Expressdienste',
                    'Paketzustellungen und Kleintransporte im Rahmen des genehmigungsfreien Güterverkehrs bis max. 3,5 t',
                    'Güternah- und Güterfernverkehr, Spezialtransporte, Lagerei und Speditionsfähigkeit, Logistik, Umweltdienstleistungen, Betreiben von umwelttechnischen Anlagen; keine Personentransporte. Des  Weiteren Beratung und Handel einschließlich der Vermittlung von Geschäften sowie Im- und Export - soweit nicht besondere Erlaubnisse erforderlich sind',
                    'a) Der Bau und der Betrieb einer normalspurigen Eisenbahn, die aus den beiden Teilstrecken Verden-Stemmen und Walsrode-Böhme besteht. Die Wiederherstellung der Schienenverbindung beider Teilstrecken wird angestrebt, b) der Betrieb,die Errichtung, der Erwerb, die Pachtung und die Verpachtung von Omnibuslinien, Gelegenheitsverkehr und Güterkraftverkehr, c) jede sonstige Förderung des Verkehrs.',
                    'das Speditionsgeschäft, insbesondere die Vermittlung von Luftfrachten sowie die gewerbsmäßige Übernahme von Gütersendungen durch Frachtführer oder durch Verfrachter auf Rechnung eines anderen im eigenen Namen zu besorgen und dies insbesondere im internationalen Bereich zu tun. Die Gesellschaft kann auch verwandte Geschäfte durchführen, insbesondere das Geschäft des Lagerhalters sowie den Im- Export von Handelsgeschäften.',
                    'Transport und Logistik.',
                    'Der Kurierdienst und der Gütertransport.',
                    ]

Werbung_Marketing = ['Werbetechnik und Dienstleistungen im Bereich der Werbung, insbesondere Fahrzeugwerbung, von der Erstellung eines Entwurfs bis zur Fertigung und dem Vertrieb',
                     'Die Herstellung, Wartung, der Vertrieb und der Handel von Computer-Software, der Handel mit Computerhardware und Zubehör, elektrischen und elektronischen Geräten und Anlagen zur         Telekommunikation, die Beratung Dritter zur Informationstechnologie sowie der Betrieb einer   Werbeagentur. Genehmigungspflichtige Geschäfte sind ausdrücklich ausgeschlossen.',
                     'Social Media und Internet-Marketing Management/ Beratung',


                     ]

Technik_Telekommunikation = ['Filmproduktion und Kameraverleih.',
                             'Großhandel mit Telekommunikationsartikeln und Büroelektronik, Hardware und Software, sowie Vermittlung von Mobilfunkverträgen.',
                             'Vertrieb elektronischer Produkte und Dienstleistungen (Services).',
                             'Die Erbringung von Fremddienstleistungen, insbesondere für Unternehmen der Energieversorgung und Telekommunikation. \n',
                             'Erstellung audiovisueller Produkte für TV und Online',
                             'Die Herstellung, Wartung, der Vertrieb und der Handel von Computer-Software, der Handel mit Computerhardware und Zubehör, elektrischen und elektronischen Geräten und Anlagen zur         Telekommunikation, die Beratung Dritter zur Informationstechnologie sowie der Betrieb einer   Werbeagentur. Genehmigungspflichtige Geschäfte sind ausdrücklich ausgeschlossen.',
                             'Einbringung oder Vermittlung von Beratungsleistungen und Dienstleistungen zur technischen Realisation von Fernseh- und Filmproduktionen sowie von audiovisuellen Veranstaltungen.',


                             ]

In [ ]:
from transformers import pipeline
classifier = pipeline("zero-shot-classification", model="MoritzLaurer/mDeBERTa-v3-base-mnli-xnli")

/usr/local/lib/python3.8/dist-packages/transformers/convert_slow_tokenizer.py:446: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [ ]:
labels = [Agrarwirtschaft, 
          Bau, 
          Chemie_Rohstoffe, 
          Dienstleistungen_Handwerk, 
          Internet, 
          Energie_Umwelt, 
          Finanzen_Versicherungen_Immobilien, 
          Metall_Elektronik,
          Freizeit,
          ÖffentlicherSektor_Bildung,
          Handel_Konsum,
          Pharma_Gesundheit,
          Tourismus_Gastronomie,
          Verkehr_Logistik,
          Werbung_Marketing,
          Technik_Telekommunikation
          ]

In [ ]:
sequence =  'ist der Betrieb von Spielhallen.'

In [ ]:
classifier(sequence, labels)

In [ ]:
model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')

In [ ]:
embedding_1 = model.encode('der Betrieb eines Fitness-Studios.', convert_to_tensor=True)
embedding_2 = model.encode('ist der Betrieb von Spielhallen.', convert_to_tensor=True)

In [ ]:
cos = torch.nn.CosineSimilarity(dim=0)
cos(embedding_1, embedding_2)

tensor(0.3974, device='cuda:0')

In [ ]:
import torch

In [ ]:
embedding_1 = model.encode('der Betrieb eines Fitness-Studios.', convert_to_tensor=True)
outputs = []
cos = torch.nn.CosineSimilarity(dim=0)


for i in labels:
  label_count = 0
  
  for j in i:
    embedding_2 = model.encode(i, convert_to_tensor=True)
    label_count += cos(embedding_1, embedding_2)

  outputs.append(str(torch.sum(label_count)))

In [ ]:
outputs

# Text Classification -> Model Creation

In [ ]:
from transformers import AutoTokenizer
from sklearn.model_selection import train_test_split

tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-large")

In [ ]:
df_labeled = pd.DataFrame()
label_count = 0
for i in labels:
  for j in i:
    temp_dict = {"text": tokenizer(j, truncation = True), "label": label_count}
    df_labeled = pd.concat([df_labeled, pd.DataFrame([temp_dict])])
  label_count += 1

In [ ]:
df_labeled = df_labeled.sample(frac = 1)
df_train = df_labeled.sample(frac=0.8)
df_test = df_labeled.drop(df_train.index)

In [ ]:
df_train

,text,label
0,Gegenstand des Unternehmens ist die Vermietung...,1
0,"[input_ids, attention_mask]",6
0,"[input_ids, attention_mask]",10
0,"[input_ids, attention_mask]",6
0,"[input_ids, attention_mask]",2
...,...,...
0,"[input_ids, attention_mask]",13
0,"Die Durchführung sämtlicher Tätigkeiten, Aufga...",1
0,"Handel mit Motorrädern, Motoradteilen und Moto...",7
0,Der Einkauf und der Verkauf von Kosmetikwaren ...,10


In [ ]:
def preprocess_function(examples):
  return tokenizer(examples, truncation=True)

In [ ]:
tokenized_industries = df_labeled["text"].map(preprocess_function)

In [ ]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
!pip install evaluate
import evaluate
accuracy = evaluate.load("accuracy")

In [ ]:
def compute_metrics(eval_pred):
  predictions, labels = eval_pred
  predictions = np.argmax(predictions, axis=1)
  return accuracy.compute(predictions=predictions, references=labels)

In [ ]:
id2label = {0: "Agrarwirtschaft", 
            1: "Bau", 
            2: "Chemie_Rohstoffe", 
            3: "Dienstleistungen_Handwerk", 
            4: "Internet",
            5: "Energie_Umwelt",
            6: "Finanzen_Versicherungen_Immobilien",
            7: "Metall_Elektronik",
            8: "Freizeit",
            9: "ÖffentlicherSektor_Bildung",
            10: "Handel_Konsum",
            11: "Pharma_Gesundheit",
            12: "Tourismus_Gastronomie",
            13: "Verkehr_Logistik",
            14: "Werbung_Marketing",
            15: "Technik_Telekommunikation"
            }
            
label2id = {"Agrarwirtschaft": 0, 
            "Bau": 1, 
            "Chemie_Rohstoffe": 2, 
            "Dienstleistungen_Handwerk": 3, 
            "Internet": 4,
            "Energie_Umwelt": 5,
            "Finanzen_Versicherungen_Immobilien": 6,
            "Metall_Elektronik": 7,
            "Freizeit": 8,
            "ÖffentlicherSektor_Bildung": 9,
            "Handel_Konsum": 10,
            "Pharma_Gesundheit": 11,
            "Tourismus_Gastronomie": 12,
            "Verkehr_Logistik": 13,
            "Werbung_Marketing": 14,
            "Technik_Telekommunikation": 15
            }

In [ ]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained("xlm-roberta-large", num_labels=16, id2label=id2label, label2id=label2id)

Some weights of the model checkpoint at xlm-roberta-large were not used when initializing XLMRobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-large and are newly initialized: ['classifier.den

In [ ]:
training_args = TrainingArguments(
  output_dir="my_awesome_model",
  learning_rate=2e-5,
  per_device_train_batch_size=2,
  per_device_eval_batch_size=2,
  num_train_epochs=50,
  weight_decay=0.01,
  evaluation_strategy="epoch",
  save_strategy="epoch",
  load_best_model_at_end=True,
  push_to_hub=False
  )

trainer = Trainer(
  model=model,
  args=training_args,
  train_dataset=X_train + y_train,
  eval_dataset=X_test + y_test,
  tokenizer=tokenizer,
  data_collator=data_collator,
  compute_metrics=compute_metrics,
  )

trainer.train()

TypeError: ignored

# Text Classification -> Classification of unlabeled data

In [ ]:
classifier = pipeline("text-classification", model=model)
classifier(text)

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline

tokenizer = AutoTokenizer.from_pretrained("sampathkethineedi/industry-classification")  
model = AutoModelForSequenceClassification.from_pretrained("sampathkethineedi/industry-classification")

industry_tags = pipeline('sentiment-analysis', model=model, tokenizer=tokenizer)

In [ ]:
industry_tags('The purchase and sale as well as the management of own real estate')

[{'label': 'Real Estate Operating Companies', 'score': 0.9997528195381165}]

Übersetzung der Texte ins Englische und anschließend Classification mithilfe des Industry Classifiers 

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_translate = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-de-en")
tokenizer_translate = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-de-en")
model_translate.to("cuda")

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline

tokenizer = AutoTokenizer.from_pretrained("sampathkethineedi/industry-classification")  
model_classification = AutoModelForSequenceClassification.from_pretrained("sampathkethineedi/industry-classification")

industry_tags = pipeline('sentiment-analysis', model=model_classification, tokenizer=tokenizer)

In [ ]:
output = industry_tags('The purchase and sale as well as the management of own real estate')

In [ ]:
output[0]["label"]

'Real Estate Operating Companies'

In [ ]:
output[0]["score"]

0.9997528195381165

In [ ]:
!pip install sacremoses

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 KB 44.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=2405b0ccd7befb9953878d02e4b365eeb70d71edb53b07e1e8b369bae7679c4f
  Stored in directory: /root/.cache/pip/wheels/82/ab/9b/c15899bf659ba74f623ac776e861cf2eb8608c1825ddec66a4
Successfully built sacremoses


In [ ]:
from transformers import FSMTForConditionalGeneration, FSMTTokenizer
mname = "facebook/wmt19-de-en"
tokenizer_trans = FSMTTokenizer.from_pretrained(mname)
model_trans = FSMTForConditionalGeneration.from_pretrained(mname)

In [ ]:
Industries = []
Scores = []

for ind in df_filtered.index:
  #input_ids = tokenizer(df_filtered['Gegenstand_oder_Geschaeftszweck 0'][ind], max_length=128, return_tensors="pt").input_ids.to("cuda")
  #sequences = model_translate.generate(input_ids)[0]
  #translation = tokenizer.batch_decode(sequences, skip_special_tokens=True)

  input_ids = tokenizer_trans.encode(df_filtered['Gegenstand'][ind], return_tensors="pt")
  outputs = model_trans.generate(input_ids)
  translation = tokenizer_trans.decode(outputs[0], skip_special_tokens=True)

  print(translation)

  output = industry_tags(translation)
  Industries.append(output[0]["label"])
  Scores.append(output[0]["score"])

/usr/local/lib/python3.8/dist-packages/transformers/generation/utils.py:1273: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 200 (`generation_config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Providing services in the areas of online marketing, SEO, SEM, internet advertising, web design and graphic design
Ownership and management of real estate and own assets.
The object of the company is the implementation and processing of transfer measures within the meaning of statutory provisions, in particular the taking over of employees from restructuring and restructuring measures of companies, the qualification, further employment, hiring, placement and integration of these employees into the labour market.
the operation of a driving and professional driver school, first aid training, the operation of visual test centres and the implementation of further training, trainings and trainings of all kinds.
Wholesale of textiles, toys and accessories.
Operation of a laundry for the production, finishing and distribution of textiles and clothing as well as the provision of consultancy and agency services typical for fashion
holding and managing own assets as well as participating in othe

In [ ]:
df_filtered['Score'] = Scores
df_filtered['Industry'] = Industry

# App mittels Gradio

# Darstellen von Graphen

In [1]:
!pip install gradio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [11]:
df_labeled_industry = pd.read_csv("/content/gdrive/MyDrive/Firmen_df_labeled.csv", index_col = 0)

In [12]:
df_labeled_industry.head()

,ID,Name,Ort,Postleitzahl,Straße,Hausnummer,Gegenstand,Stammkapital,Gesellschaftsform,Score,Industry
0,1.0,09 Media GmbH,Regensburg,93059.0,Im Gewerbepark,A10,Anbietung von Dienstleistungen in den Bereiche...,25000.0,Gesellschaft mit beschränkter Haftung,0.960162,Advertising
1,2.0,1. Immobilienverwaltung Schülein GmbH,Wittelshofen,91749.0,Untermichelbach,50,Der Besitz und die Verwaltung von Immobilien u...,25000.0,Gesellschaft mit beschränkter Haftung,0.999847,Real Estate Operating Companies
4,5.0,10 k Transfer GmbH,Speyer,67346.0,Stuhlbrudergasse,5,Gegenstand des Unternehmens ist die Durchführu...,25000.0,Gesellschaft mit beschränkter Haftung,0.987049,Human Resource & Employment Services
6,7.0,123fahrschule Ruhrgebiet-Süd GmbH,Köln,50968.0,Klopstockstraße,1,der Betrieb einer Fahr- und Berufskraftfahrers...,25000.0,Gesellschaft mit beschränkter Haftung,0.502101,Diversified Support Services
7,8.0,123Fashiontrade.com GmbH,Neuss,41460.0,"Breslauer Straße 1, Haus Milano Raum",205,"Großhandel mit Textilien, Spielwaren und Acces...",25000.0,Gesellschaft mit beschränkter Haftung,0.890654,Trading Companies & Distributors


In [13]:
unique_industries = df_labeled_industry["Industry"].unique()
print(unique_industries)
print(len(unique_industries))

['Advertising' 'Real Estate Operating Companies'
 'Human Resource & Employment Services' 'Diversified Support Services'
 'Trading Companies & Distributors' 'Asset Management & Custody Banks'
 'Commodity Chemicals' 'Research & Consulting Services'
 'Personal Products' 'Leisure Products' 'Movies & Entertainment'
 'Apparel, Accessories & Luxury Goods' 'Property & Casualty Insurance'
 'Investment Banking & Brokerage' 'Technology Distributors' 'Homebuilding'
 'Casinos & Gaming' 'Specialty Chemicals' 'IT Consulting & Other Services'
 'Specialty Stores' 'Health Care Facilities'
 'Electrical Components & Equipment' 'Construction & Engineering'
 'Internet & Direct Marketing Retail' 'Industrial Machinery'
 'Systems Software' 'Building Products' 'Aerospace & Defense'
 'Auto Parts & Equipment' 'Environmental & Facilities Services' 'Steel'
 'Restaurants' 'Health Care Services' 'Oil & Gas Refining & Marketing'
 'Consumer Finance' 'Internet Services & Infrastructure'
 'Electric Utilities' 'Packaged F

In [14]:
unique_industries = unique_industries.tolist()

In [15]:
import plotly.express as px
import gradio as gr
import plotly.express as px
import matplotlib.pyplot as plt


In [16]:
def Dropdown_list(df_labeled_industry):
    return df_labeled_industry['Industry']

In [17]:
def plot_barchart(df_labeled_industry):
    fig = plt.figure()
    plt.boxplot(x=df_labeled_industry['Industry']) #, y=df_pens['bill_depth_mm'])
    return fig

In [18]:
demo = gr.Blocks()

options = unique_industries
with demo:
  text_options = gr.Dropdown(options, label='Enter an industry')
  outputs=plot_barchart,
  title="Barchart depicting the different industries and the number of associated companies",
  description="Check for the different industries"
  
demo.launch(debug=False)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

In [19]:
#'gr.Interface(fn=plot_barchart, 
    inputs=gr.inputs.Textbox(label='What industry are you looking for?'),
    outputs=['plot'],
    title="Barchart depicting the different industries and the number of associated companies",
    description="Check for the different industries").launch(debug=True)

SyntaxError: ignored